# Heat equation example in 1D

One of the simplest examples of solving PDEs with neural networks is considered in this notebook. A problem is set up in such a way that it admits an exact solution, which can be compared to a PINN approximation. The heat transfer example is based on the **one-dimensional heat equation**
$$
\frac{\partial u}{\partial t} - \alpha \frac{\partial^2 u}{\partial x^2} = 0, \quad
t \in [0, T], \quad x \in [0, L].
$$
Dirichlet boundary conditions $u(t, 0) = u(t, L) = 0$ for $t \in [0, T]$ and initial conditions $u(0, x) = u_0(x) = \sin \left( \frac{n \pi x}{L} \right)$ for $x \in [0, L]$ and a certain $n \in \{1, 2, \ldots\}$ are imposed. Through **separation of variables** one can obtain the factorized solution as
$$
u(t, x) = \sin \left( \frac{n \pi}{L} x \right) \exp \left( -\frac{n^2 \pi^2}{L^2} \alpha t \right).
$$

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append('..')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

from utils import (
    HeatConduction1D,
    PINN,
    train_pinn,
    make_colors
)

In [ ]:
_ = torch.manual_seed(123456789) # set random seed manually

## Problem setup

As a first step, we formulate a specific heat transfer problem by setting the defining physical parameters. An instance `simple_problem` of the class `HeatConduction1D` is created to that end.

In [ ]:
alpha = 0.07 # thermal diffusivity
length = 1.0 # rod length
maxtime = 1.0 # time length
n = 3 # initial condition

simple_problem = HeatConduction1D(
    alpha=alpha,
    length=length,
    maxtime=maxtime,
    n=n
)

## Exact solution

The problem defined above admits an analytical solution. It can be computed by calling the class instance we just created for certain time and location inputs. This simply looks like `u_values = simple_problem(t=t, x=x)` in the code.

In [ ]:
t_values = torch.linspace(0, maxtime, 1001)
x_values = torch.linspace(0, length, 1001)

t = t_values.view(-1, 1) # (time, 1)
x = x_values.view(1, -1) # (1, space)

u_values = simple_problem(t=t, x=x) # (time, space)

In the cells below, the exact solution from above is visualized in two different ways.

In [ ]:
tids = (0, 50, 100, 150, 200, -1)
colors = make_colors(len(tids), seq_cm=plt.cm.viridis_r, ensure_seq=True)

fig, ax = plt.subplots(figsize=(6, 4))
for tidx, color in zip(tids, colors):
    ax.plot(
        x_values.numpy(), u_values[tidx,:].numpy(),
        color=color, alpha=0.8,
        label='t={:.2f}'.format(t_values[tidx].item())
    )
ax.set(xlabel='x', ylabel='u(t, x)')
ax.set_xlim((x_values.min(), x_values.max()))
ax.legend()
ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
ax.set_axisbelow(True)
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
img = ax.imshow(
    u_values.numpy().T,
    cmap='PRGn',
    aspect='auto',
    interpolation='bilinear',
    vmin=np.round(u_values.min()),
    vmax=np.round(u_values.max()),
    origin='lower',
    extent=(0, maxtime, 0, length)
)
ax.set(xlabel='t', ylabel='x')
fig.colorbar(img, ax=ax)
fig.tight_layout()

## PINN approximation

An approximation to the exact solution is now computed. We therefore instantiate a `PINN`-object that constructs a feedforward NN as the prediction model and features dedicated methods for computing the losses. A small number of architectural parameters and loss weights have to be set.

In [ ]:
num_inputs = 2 # number of inputs
num_outputs = 1 # number of outputs
num_hidden = (32, 32, 32) # number of hidden neurons
activation = 'tanh' # activation function

pde_weight = 1.0 # weight of the PDE loss
bc_weight = 1.0 # weight of the BC loss
ic_weight = 1.0 # weight of the IC loss
reduction = 'mean' # loss reduction mode

pinn = PINN(
    num_inputs,
    num_outputs,
    num_hidden=num_hidden,
    activation=activation,
    pde_weight=pde_weight,
    bc_weight=bc_weight,
    ic_weight=ic_weight,
    reduction=reduction,
    alpha=alpha,
    length=length,
    maxtime=maxtime,
    n=n
)

Two sets of collocation points are randomly sampled for training and validation purposes. They remain fixed throughout the training once they are created. It is noted that, of course, one could randomly resample during training or opt for a non-random selection as well.

In [ ]:
train_nums = {
    'num_pde': 5000, # number of train samples for PDE loss
    'num_bc': 500, # number of train samples for BC loss
    'num_ic': 500 # number of train samples for IC loss
}

val_nums = {
    'num_pde': 1000, # number of val. samples for PDE loss
    'num_bc': 100, # number of val. samples for BC loss
    'num_ic': 100 # number of val. samples for IC loss
}

train_colloc = pinn.make_collocation(**train_nums, random=True)
val_colloc = pinn.make_collocation(**val_nums, random=True)

The function `train_pinn` implements a non-batched PINN training scheme for fixed collocation points. It runs on the CPU due to the problem being rather small-scale after all. No data-based regression loss is used here, such that the training proceeds by minimizing only a physics-based loss function.

In [ ]:
num_epochs = int(1e04) # number of training epochs

optimizer = torch.optim.Adam(pinn.parameters(), lr=0.001)

history = train_pinn(
    pinn,
    optimizer,
    num_epochs,
    train_colloc,
    val_colloc,
    print_every=100
)

Let us have a look at the learning curves.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(history['train_loss'], alpha=0.7, label='train')
ax.plot(history['val_loss'], alpha=0.7, label='val')
ax.set(xlabel='epoch', ylabel='physics loss')
ax.set_xlim((0, max(len(history['train_loss']), len(history['val_loss'])) - 1))
ax.set_yscale('log')
ax.legend()
ax.grid(visible=True, which='both', color='gray', alpha=0.2, linestyle='-')
ax.set_axisbelow(True)
fig.tight_layout()

In order to assess the quality of the trained PINN, one can compare its predictions to the exact solution. This requires to evaluate the approximation on a grid of input values.

In [ ]:
t_grid, x_grid = torch.meshgrid(t_values, x_values, indexing='ij')

with torch.no_grad():
    pred_values = pinn(t=t_grid.reshape(-1), x=x_grid.reshape(-1)) # (time * space, 1)
    pred_values = pred_values.reshape(*t_grid.shape) # (time, space)

It is finally time to plot the obtained approximation. This is done in the same two ways as for the analytical solution.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
for tidx, color in zip(tids, colors):
    ax.plot(
        x_values.numpy(), pred_values[tidx,:].numpy(),
        color=color, alpha=0.8,
        label='t={:.2f}'.format(t_values[tidx].item())
    )
ax.set(xlabel='x', ylabel='u(t, x)')
ax.set_xlim((x_values.min(), x_values.max()))
ax.legend()
ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
ax.set_axisbelow(True)
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
img = ax.imshow(
    pred_values.numpy().T,
    cmap='PRGn',
    aspect='auto',
    interpolation='bilinear',
    vmin=np.round(pred_values.min()),
    vmax=np.round(pred_values.max()),
    origin='lower',
    extent=(0, maxtime, 0, length)
)
ax.set(xlabel='t', ylabel='x')
fig.colorbar(img, ax=ax)
fig.tight_layout()

Well, that looks quite OK!